In [1]:
%load_ext autoreload
%autoreload 2

# general
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.interpolate import interp1d

# plotting
import matplotlib.pyplot as plt
from ipywidgets import interact
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# fitting
from sklearn.metrics import r2_score
from scipy.optimize import minimize
from functools import partial
import multiprocess as mp
import matplotlib.patches as mpatches

# custom
from reflectance import plotting, spectrum_utils, optimisation_pipeline, file_ops

In [28]:
config_dict = {
    "processing": 
    {
        'aop_group_num': 1,
        'nir_wavelengths': [750, 1100],
        'sensor_range': [450, 690],
        'endmember_dimensionality_reduction': 'mean',
        'endmember_normalisation': False,
        'endmember_class_schema': 'individual',
        'spectra_normalisation': False,
        'endmember_source': 'data/AOP_models/Rb_model_single_coefficient.txt'
    },
    "simulation":
    {
        "type": "spread",
        "N": 10,
        "Rb_vals": [0,1,0],
        "n_depths": 10,
        "depth_lims": [0, 20],
        "n_ks": 10,
        "k_lims": [0, 0.4],
        "n_bbs": 10,
        "bb_lims": [0.01, 0.03],
        "n_noise_levels": 10,
        "noise_lims": [0.001, 0],
        "noise_ind": 0
    },
    "fitting":
    {
        'objective_fn': 'og_r2',
        'Rb_init': 0,
        'bb_bounds': (0, 0.41123),
        'Kd_bounds': (0.01688, 3.17231),
        'H_bounds': (0, 20),
        'endmember_bounds': (0, 1),
        'solver': 'L-BFGS-B',
        'tol': 1e-9
    }}
g_cfg, _ = file_ops.instantiate_single_configs_instance(run_ind = 0)
cfg = file_ops.RunOptPipeConfig(config_dict)

pipe = optimisation_pipeline.OptPipe(g_cfg, cfg)
fit_results = pipe.run()

ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.

In [18]:
g_cfg, _ = file_ops.instantiate_single_configs_instance(run_ind = 0)
g_cfg

GlobalOptPipeConfig(spectra_source='prism', spectra_fp=PosixPath('/Users/rt582/Library/CloudStorage/OneDrive-UniversityofCambridge/cambridge/phd/coralreflections/data/CORAL_validation_spectra.csv'), spectral_library_fp=PosixPath('/Users/rt582/Library/CloudStorage/OneDrive-UniversityofCambridge/cambridge/phd/coralreflections/reflectance/resources/spectral_library_clean_v3_PRISM_wavebands.csv'), validation_data_fp=PosixPath('/Users/rt582/Library/CloudStorage/OneDrive-UniversityofCambridge/cambridge/phd/coralreflections/data/CORAL_validation_data.csv'), save_fits=False, endmember_map={'algae_crustose_coralline': ['algCCA'], 'algae_fleshy_brown': ['algMacBrn'], 'algae_fleshy_green': ['algMacGrn', 'algMacMix', 'algMacUnk', 'algCyano'], 'algae_fleshy_red': ['algMacRed'], 'algae_turf': ['algTurf'], 'coral_blue': [], 'coral_brown': ['crlCoral'], 'mud': ['sedMud'], 'octocoral': ['othOcto'], 'sand': ['sedSand', 'crlBleach', 'sedLimest', 'sedRubble'], 'seagrass': ['othSeagr']}, endmember_schema={

# Pipeline

In [347]:
fitted_df = fit_results.fitted_spectra
wvs = fitted_df.columns
lim_prism_spectra = preprocess_prism_spectra(spectrum_utils.load_spectra(), nir_wavelengths = [750,1100], sensor_range = [450,690]).loc[:len(fitted_df)-1, :]

In [ ]:
# plot fitted spectra
i = 10
true_spectrum = lim_prism_spectra.iloc[i]
fitted_spectrum = fit_results.fitted_spectra.iloc[i]

plt.plot(wvs, fitted_spectrum, label='fitted')
plt.plot(wvs, true_spectrum, label='true')
plt.legend()
from sklearn.metrics import r2_score
# print("calculated r2:", r2_score(lim_prism_spectra.iloc[i], fit_results.fitted_spectra.iloc[i]))
print("recorded r2:", fit_results.metrics.r2.iloc[i], "sa:", fit_results.metrics.spectral_angle.iloc[i])

In [ ]:
# SMOOTHING AND DERIVATIVES
from scipy.signal import savgol_filter

window_length = 11
order = 5

# differentiate spectra
diff_fitted = np.diff(fitted_spectrum)
diff_true = np.diff(true_spectrum)
smooth_diff_fitted = savgol_filter(diff_fitted, window_length, polyorder=order)
smooth_diff_true = savgol_filter(diff_true, window_length, polyorder=order)

d1_wvs = (wvs[1:] + wvs[:-1]) / 2
d2_wvs = (d1_wvs[1:] + d1_wvs[:-1]) / 2

fig, ax = plt.subplots(2, 2, figsize=(12, 4))


ax[0,0].plot(d1_wvs, diff_fitted, label='$\\Delta$ fitted')
ax[0,0].plot(d1_wvs, diff_true, label='$\\Delta$ true')
ax[1,0].plot(d2_wvs, np.diff(diff_fitted), label='$\\Delta$ fitted')
ax[1,0].plot(d2_wvs, np.diff(diff_true), label='$\\Delta$ true')

ax[0,1].plot(d1_wvs, smooth_diff_fitted, label='$\\Delta$ fitted')
ax[0,1].plot(d1_wvs, smooth_diff_true, label='$\\Delta$ true')
ax[1,1].plot(d2_wvs, np.diff(smooth_diff_fitted), label='$\\Delta$ fitted')
ax[1,1].plot(d2_wvs, np.diff(smooth_diff_true), label='$\\Delta$ true')

[ax_.legend() for ax_ in ax.flatten()];

In [ ]:
fits = pd.concat([fit_results.fitted_params, fit_results.metrics], axis=1)
validation_data = pd.read_csv(file_ops.DATA_DIR_FP / "CORAL_validation_data_v2.csv")

fig, ax = plt.subplots()
ma = ax.scatter(validation_data.Depth[:len(fits)], fits.H, alpha=0.4, c=fits.r2, edgecolor='k', lw=0.5)

# calculate fits
p = np.polyfit(validation_data.Depth[:len(fits)], fits.H.astype(float), 1)
pred = np.polyval(p, validation_data.Depth[:len(fits)])
r2 = r2_score(validation_data.Depth[:len(fits)], pred)
ax.plot(validation_data.Depth[:len(fits)], pred, color='r', ls='--', label="best fit")
# formatting
ax.set_xlim(left=0)
ax.set_ylim(bottom=0, top=40)
ax.plot(ax.get_xlim(),ax.get_xlim(), color='k', label='1:1')
ax.set_title(f"r$^2$: {r2:.5f}")
ax.set_xlabel("True depth (m)")
ax.set_ylabel("Retrieved depth (m)")
# ax.set_aspect('equal')
fig.colorbar(ma, label="r$^2$ value")
ax.legend()


In [ ]:
plt.hist(fits.r2, bins=50)
plt.yscale('log')

In [ ]:
# view fits
test_spectra = pd.read_csv(file_ops.RESULTS_DIR_FP / "fits/sim_spectra_1.csv")
wvs = test_spectra.columns[4:].astype(float)
plt.plot(wvs, test_spectra.iloc[:,4:].values.T, color='k', alpha=0.01);

In [ ]:
test_fits = pd.read_csv(file_ops.RESULTS_DIR_FP / "fits/fit_results_1.csv", skiprows=1)
wvs = test_fits.iloc[:,6:-2].columns.astype('float')
plt.plot(wvs, test_fits.iloc[:,6:-2].values.T, color='k', alpha=0.01);

In [ ]:
fits = test_fits
validation_data = pd.read_csv(file_ops.DATA_DIR_FP / "CORAL_validation_data_v2.csv")


fig, ax = plt.subplots()
ma = ax.scatter(validation_data.Depth[:len(fits)], fits.H, alpha=0.4, c=fits.r2, edgecolor='k', lw=0.5)
ax.plot(ax.get_xlim(),ax.get_xlim(), color='k', label='1:1')

# calculate fits
p = np.polyfit(validation_data.Depth[:len(fits)], fits.H, 1)
pred = np.polyval(p, validation_data.Depth[:len(fits)])
r2 = r2_score(validation_data.Depth[:len(fits)], pred)
ax.plot(validation_data.Depth[:len(fits)], pred, color='r', ls='--', label="best fit")
# formatting
ax.set_title(f"r$^2$: {r2:.5f}")
ax.set_xlabel("True depth (m)")
ax.set_ylabel("Retrieved depth (m)")
ax.set_ylim(0,40)
# fig.colorbar(ma, label="r$^2$ value")
ax.legend()


In [ ]:
# Create a scatter plot of retrieved vs actual depth
scatter = go.Scatter(
    x=validation_data.Depth[:len(fits)],
    y=fits.H,
    mode='markers',
    marker=dict(
        size=10,
        color=fits.r2,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="whole-spectrum r<sup>2</sup> value"),
        line=dict(width=0.5, color='#000000'),
        opacity=0.5,
        # assign hover labels the respective values in validation_data
      ),
        name='Data points',
        hovertext=[f"K: {K:.2f}, bb: {bb:.2f}, r<sup>2</sup>: {r2:.5f}, sa: {sa:.5f}, sample: {i}" for K, bb, r2, sa, i in zip(fits.K, fits.bb, fits.r2, fits.spectral_angle, fits.index)],
    # hoverinfo="text"
)

p = np.polyfit(validation_data.Depth[:len(fits)], fits.H, 1)
pred = np.polyval(p, validation_data.Depth[:len(fits)])
# THIS LOOKS WRONG
best_fit_line = go.Scatter(
    x=validation_data.Depth[:len(fits)],
    y=pred,
    mode='lines',
    line=dict(color='red', dash='dash'),
    name=f"Best fit: {p[0]:.2f}x + {p[1]:.2f}",
)
one_to_one_line = go.Scatter(
    x=[min(validation_data.Depth[:len(fits)]), max(validation_data.Depth[:len(fits)])],
    y=[min(validation_data.Depth[:len(fits)]), max(validation_data.Depth[:len(fits)])],
    mode='lines',
    line=dict(color='black'),
    name='1:1'
)

plot = True
if plot:
    fig = make_subplots()
    fig.add_trace(scatter)
    fig.add_trace(best_fit_line)
    fig.add_trace(one_to_one_line)

    # Update layout
    fig.update_layout(
        title=f"r<sup>2</sup>: {r2_score(validation_data.Depth[:len(fits)], fits.H):.5f}",
        xaxis_title="True depth (m)",
        yaxis_title="Retrieved depth (m)",
        xaxis=dict(range=[min(validation_data.Depth[:len(fits)]), max(validation_data.Depth[:len(fits)])], scaleanchor='y', scaleratio=1),
        yaxis=dict(range=[min(validation_data.Depth[:len(fits)]), max(validation_data.Depth[:len(fits)])], scaleanchor='x', scaleratio=1),
        legend=dict(x=0.02, y=0.98),   
    )

    fig.update_yaxes(
        scaleanchor='x',
        scaleratio=1,
    )

    # show the figure
    fig.show()

In [ ]:
fits.iloc[:,6:-2].columns.astype('float')

In [ ]:
fits.iloc[i,:3+len(endmember_array)]

In [ ]:
fits.iloc[i]
# looks like the calculate of r2 being stored here is wrong

# 

In [ ]:
glob_cfg, l_cfg = file_ops.instantiate_single_configs_instance(run_ind = 0)
l_cfg

In [ ]:
glob_cfg, _ = file_ops.instantiate_single_configs_instance(run_ind = 0)

endmember_class_schema = "three_endmember"
endmembers = optimisation_pipeline.GenerateEndmembers(
    endmember_class_map = glob_cfg.endmember_schema[endmember_class_schema],
    endmember_dimensionality_reduction = "mean"
    ).generate_endmembers()
endmembers


In [ ]:
prism_spectra = spectrum_utils.crop_spectra_to_range(spectrum_utils.load_spectra(), [min(wvs), max(wvs)])
plt.plot(wvs, prism_spectra.values.T, color='k', alpha=0.01);
# spectra.loc[min(wvs):max(wvs)]


i = 582

sensor_range = [450, 690]

AOP_model = spectrum_utils.load_aop_model(aop_group_num=1).loc[sensor_range[0]:sensor_range[1]]
wvs = AOP_model.index
AOP_sub = AOP_model.loc[wvs]
AOP_args = (AOP_sub.bb_m.values, AOP_sub.bb_c.values, AOP_sub.Kd_m.values, AOP_sub.Kd_c.values)

# read in spectral library
f = file_ops.RESOURCES_DIR_FP / "spectral_library_clean_v3_PRISM_wavebands.csv"
df = pd.read_csv(f, skiprows=1).set_index('wavelength')
df.columns = df.columns.astype(float)
df = df.astype(float)

# generate endmembers
endmember_class_schema = "three_endmember"
glob_cfg, _ = file_ops.instantiate_single_configs_instance(run_ind = 0)
# generate endmember array
endmember_array = optimisation_pipeline.GenerateEndmembers(
    endmember_class_map=glob_cfg.endmember_schema[endmember_class_schema],
    endmember_dimensionality_reduction="mean",
).generate_endmembers()
# crop to sensor range
endmember_array = endmember_array.loc[:,sensor_range[0]:sensor_range[1]]

# three_em_cats = {
#     'algae': ['algae_fleshy_brown', 'algae_fleshy_green', 'algae_fleshy_red', 'algae_turf', 'seagrass', 'algae_crustose_coralline'],
#     'coral': ['coral_blue', 'coral_brown', 'octocoral'],
#     'sand': ['sand', 'mud']
# }

# three_endmembers = {}
# for cat in three_em_cats:
#     ind = df.index.isin(three_em_cats[cat])
#     # select all spectra in category and calculate mean spectrum
#     three_endmembers[cat] = df.loc[ind].mean(axis=0).loc[sensor_range[0]:sensor_range[1]]
# # create array of average spectrum for each category
# three_endmember_array = np.array([spectrum.values for spectrum in three_endmembers.values()])

endmember_array = three_endmember_array
endmember_cats = three_em_cats

fit = fits.iloc[i,:3+len(endmember_array)]
true_spectrum = prism_spectra.iloc[i,:]
# plot problem fits
plotting.plot_single_fit(fit, true_spectrum, AOP_args, endmember_array, endmember_cats.keys())

In [ ]:
# plot r2
plt.hist(test_fits.spectral_angle, bins=50);

In [ ]:
out, _ = spectrum_utils.spread_simulate_spectra(wvs, three_endmember_array, AOP_args, N=200, Rb_vals=(0.4, 0.4, 0.2))
plt.plot(out.reshape(-1, out.shape[-1]).T, color='k', alpha=0.01);

In [ ]:
glob_cfg, run_cfg = file_ops.instantiate_single_configs_instance(run_ind = 0)

pipe = optimisation_pipeline.OptPipe(glob_cfg, run_cfg)
pipe.run()

# Manual fitting analysis

In [ ]:
# SETUP
prism_spectra = spectrum_utils.crop_spectra_to_range(spectrum_utils.load_spectra(), [min(wvs), max(wvs)])
# plt.plot(wvs, prism_spectra.values.T, color='k', alpha=0.01);

AOP_model = spectrum_utils.load_aop_model(aop_group_num=1).loc[sensor_range[0]:sensor_range[1]]
wvs = AOP_model.index
AOP_sub = AOP_model.loc[wvs]
AOP_args = (AOP_sub.bb_m.values, AOP_sub.bb_c.values, AOP_sub.Kd_m.values, AOP_sub.Kd_c.values)

# read in spectral library
f = file_ops.RESOURCES_DIR_FP / "spectral_library_clean_v3_PRISM_wavebands.csv"
df = pd.read_csv(f, skiprows=1).set_index('wavelength')
df.columns = df.columns.astype(float)
df = df.astype(float)

# GENERATE ENDMEMBERS
glob_cfg = file_ops.GlobalOptPipeConfig(file_ops.read_yaml(file_ops.CONFIG_DIR_FP / "glob_cfg.yaml"))
sensor_range = [450, 690]

endmember_class_schema = "three_endmember"
endmember_df = optimisation_pipeline.GenerateEndmembers(
    endmember_class_map = glob_cfg.endmember_schema[endmember_class_schema],
    endmember_dimensionality_reduction = "mean",
    endmember_normalisation = False
).generate_endmembers()
endmember_df = spectrum_utils.crop_spectra_to_range(endmember_df, sensor_range)
endmember_array = endmember_df.values

# generate simulated data
sim_spectra, sim_metadata = spectrum_utils.spread_simulate_spectra(wvs, three_endmember_array, AOP_args, N=200, Rb_vals=(0.4, 0.4, 0.2))

In [ ]:
sim_spectra

In [ ]:
# for a given validation spectrum, minimise with respect to various objective functions
i = 6
obs_spectra = prism_spectra

# depth_bounds = (1,2)
depth_bounds = (0,50)

of = spectrum_utils.sa_objective_unity_fn
# minimise
endmember_bounds = (0, 1)
fit = minimize(
            of,
            # initial parameter values
            x0=[0.1, 0.1, 0] + [0.0001] * len(endmember_array),
            # extra arguments passsed to the object function (and its derivatives)
            args=(obs_spectra.loc[i], # spectrum to fit (obs)
                  *AOP_args,    # wavelength-dependent backscatter and attenuation coefficients (bb_m, bb_c, Kd_m, Kd_c)
                  endmember_array  # typical end-member spectra
                  ),
            # constrain values
            bounds=[(0, 0.41123), (0.01688, 3.17231), depth_bounds] + [endmember_bounds] * len(endmember_array), # may not always want to constrain this (e.g. for PCs)
            method="L-BFGS-B",
            tol=1e-15
            )

true_spectrum = obs_spectra.iloc[i,:]
# cast to same format as saved
fit = pd.Series(fit.x[:])

plotting.plot_single_fit(fit, true_spectrum, AOP_args, endmember_array, endmember_df.index)
print(fit)
print("sum of endmember contribution:", sum(fit[3:]))

In [ ]:
# highlight discrepensies wrt an objective function
kernel_width = 20
kernel_displacement = 20
wvs = true_spectrum.index

fit_spectrum = spectrum_utils.generate_spectrum(fit, wvs, endmember_array, AOP_args)
# combine the observed and fitted spectra in a numpy array
comp_spectra = np.array([true_spectrum.values, fit_spectrum])
metric = spectrum_utils.calc_ssq

wv_pairs, mean_corrs = spectrum_utils.calc_rolling_similarity(wvs, comp_spectra, kernel_width, kernel_displacement, metric)

plotting.plot_rolling_spectral_similarity(wv_pairs, mean_corrs, wvs, comp_spectra)

# Pipeline automation

In [5]:
config_params_dict = {
    'processing': {
        'aop_group_num': [1, 2, 3],
        'nir_wavelengths': [[750, 1100]],
        'sensor_range': [[450, 690]],
        'endmember_dimensionality_reduction': ["mean", "median", "pca", "nmf", "ica", "svd"],
        'endmember_normalisation': ["minmax", "zscore", "robust", "maxabs"],
        'endmember_class_schema': ["four_endmember", "all", "inorganic_organic"],
        'spectra_normalisation': ["minmax", "zscore", "robust", "maxabs"]
    },
    "simulation": {
        "type": ["spread"],
        "N": [100],
        "Rb_vals": [(0,1)],
        "n_depths": [10],
        "depth_lims": [(0, 10)],
        "n_ks": [10],
        "k_lims": [(0, 0.4)],
        "n_bbs": [10],
        "bb_lims": [(0.01, 0.03)],
        "n_noise_levels": [10],
        "noise_lims": [(0.001, 0)],
        "noise_ind": [0] 
    },
    'fitting': {
        'objective_fn': ["r2", "spectral_angle"],
        'bb_bounds': [(0, 0.41123), (0, 0.2)],
        'Kd_bounds': [(0.01688, 3.17231), (0.01688, 1.5)],
        'H_bounds': [(0, 10), (0, 40), 
                    #  (0, 20), (0, 30)
                     ],
        'solver': ['L-BFGS-B'], 
        # 'Powell', 'CG', 'BFGS', 'Newton-CG', 'L-BFGS-B', 'TNC', 'COBYLA', 'COBYQA', 'SLSQP', 'trust-constr', 'dogleg', 'trust-ncg', 'trust-exact', 'trust-krylov'],
        'tol': [
            # 0.0, 1e-3, 
            1e-6, 1e-9]
    }
}


run_cfgs = file_ops.generate_config_dicts(config_params_dict)
glob_cfg = file_ops.read_yaml(file_ops.CONFIG_DIR_FP / "glob_cfg.yaml")

In [ ]:
# checking for repeat results file
results_fp = file_ops.RESULTS_DIR_FP / "results_summary.csv"
# write header if new file
if not results_fp.exists():
    pass
else:
    runs = pd.read_csv(results_fp)
    irrelevant_sub_columns = ['datetime (UTC)', 'save_fits', 'count', 'mean', 'std', 'min']
    irrelevant_columns = ["spectral_angle", "r2", "rmse", "mean_abs_dev", "median_abs_dev"]
    # drop any columns which contain strings included in "irrelevant_columns"
    runs = runs.drop(columns=[col for col in runs.columns if any([sub in col for sub in irrelevant_columns])])
    # drop any columns for which value in first row is in "irrelevant_sub_columns"
    runs = runs.drop(columns=[col for col in runs.columns if runs[col].iloc[0] in irrelevant_sub_columns])
    
    # check against values in config file
    for col in runs.columns:
        if col in glob_cfg:
            if glob_cfg[col] == list(runs[col].unique()):
                runs = runs.drop(columns=col)
        if col in cfg:
            if cfg[col] == list(runs[col].unique()):
                runs = runs.drop(columns=col)
    # if all columns are irrelevant (i.e. no new information), skip
    if len(runs.columns) == 0:
        pass
    
runs

In [ ]:
run_cfgs[0]

In [ ]:
optimisation_pipeline.run_pipeline(glob_cfg, run_cfgs[:3])